Interesting and quite simple problem to cound 1 bits in buffer.

I think you might get different results if your data is not long enough. I got these from some links and then modified to use numpy and bytearray. These are generally untested.

http://www.valuedlessons.com/2009/01/popcount-in-python-with-benchmarks.html

http://stackoverflow.com/questions/9829578/fast-way-of-counting-bits-in-python

http://blog.philippklaus.de/2014/10/counting-bits-set-to-1-in-bytes-with-python-popcount-or-hamming-weight/

https://en.wikipedia.org/wiki/Hamming_weight#Efficient_implementation

http://www.expobrain.net/2013/07/29/hamming-weights-python-implementation/

http://stackoverflow.com/questions/8220801/how-to-use-timeit-module

http://graphics.stanford.edu/~seander/bithacks.html

In [1]:
import random, struct
import numpy as np
import gmpy2
from gmpy2 import mpz
# not meant to be random
random.seed(1)
d = bytearray([random.randint(0,255) for i in range(4096)])
print(len(d))
v = np.frombuffer(d, dtype=np.uint32)
print(v.shape, v[0])
v = np.frombuffer(d, dtype=np.uint64)*mpz(1)
print(v.shape, v[0], gmpy2.popcount(v[0]))

def count1s64(d):
    v = np.frombuffer(d, dtype=np.uint64)
    v = np.bitwise_and(v, 0x5555555555555555) + np.right_shift(np.bitwise_and(v, 0xAAAAAAAAAAAAAAAA), 1)
    v = np.bitwise_and(v, 0x3333333333333333) + np.right_shift(np.bitwise_and(v, 0xCCCCCCCCCCCCCCCC), 2)
    v = np.bitwise_and(v, 0x0F0F0F0F0F0F0F0F) + np.right_shift(np.bitwise_and(v, 0xF0F0F0F0F0F0F0F0), 4)
    v = np.bitwise_and(v, 0x00FF00FF00FF00FF) + np.right_shift(np.bitwise_and(v, 0xFF00FF00FF00FF00), 8)
    v = np.bitwise_and(v, 0x0000FFFF0000FFFF) + np.right_shift(np.bitwise_and(v, 0xFFFF0000FFFF0000), 16)
    v = np.bitwise_and(v, 0x00000000FFFFFFFF) + np.right_shift(v, 32)
    return v.sum()
v = np.frombuffer(d, dtype=np.uint64)
print(count1s64(d))

4096
(1024,) 1015160900
(512,) 14047262688061562948 29
16203


In [2]:
import numpy as np
import gmpy2
from gmpy2 import mpz

import random, struct

class popcount:
    POPCOUNT_TABLE16 = np.zeros(2**16, dtype=int) #has to be an array

    for index in range(len(POPCOUNT_TABLE16)):
        POPCOUNT_TABLE16[index] = (index & 1) + POPCOUNT_TABLE16[index >> 1]

    def popcount32_table16(v):
        return (popcount.POPCOUNT_TABLE16[ v        & 0xffff] +
                popcount.POPCOUNT_TABLE16[(v >> 16) & 0xffff])

    def count1s(d):
        v = np.frombuffer(d, dtype=np.uint32)
        return popcount.popcount32_table16(v).sum()

    POPCOUNT_TABLE16b = np.zeros(2**16, dtype=np.byte) #has to be an array

    for index in range(len(POPCOUNT_TABLE16b)):
        POPCOUNT_TABLE16b[index] = (index & 1) + POPCOUNT_TABLE16b[index >> 1]

    def popcount32_table16b(v):
        return (popcount.POPCOUNT_TABLE16b[ v        & 0xffff] +
                popcount.POPCOUNT_TABLE16b[(v >> 16) & 0xffff])

    def count1sb(d):
        v = np.frombuffer(d, dtype=np.uint32)
        return popcount.popcount32_table16b(v).sum()

    m1  = 0x5555555555555555
    m2  = 0x3333333333333333
    m4  = 0x0f0f0f0f0f0f0f0f
    m8  = 0x00ff00ff00ff00ff
    m16 = 0x0000ffff0000ffff
    m32 = 0x00000000ffffffff
    h01 = 0x0101010101010101

    def count1s64(d):
        v = np.frombuffer(d, dtype=np.uint64)
        v = np.bitwise_and(v, popcount.m1) + np.right_shift(np.bitwise_and(v, 0xAAAAAAAAAAAAAAAA), 1)
        v = np.bitwise_and(v, popcount.m2) + np.right_shift(np.bitwise_and(v, 0xCCCCCCCCCCCCCCCC), 2)
        v = np.bitwise_and(v, popcount.m4) + np.right_shift(np.bitwise_and(v, 0xF0F0F0F0F0F0F0F0), 4)
        v = np.bitwise_and(v, popcount.m8) + np.right_shift(np.bitwise_and(v, 0xFF00FF00FF00FF00), 8)
        v = np.bitwise_and(v, popcount.m16) + np.right_shift(np.bitwise_and(v, 0xFFFF0000FFFF0000), 16)
        v = np.bitwise_and(v, popcount.m32) + np.right_shift(v, 32)
        return v.sum()
    
    def count1s64_3(d):
        v = np.frombuffer(d, dtype=np.uint64)
        v = v - np.bitwise_and(np.right_shift(v, 1), popcount.m1)
        v = np.bitwise_and(v, popcount.m2) + np.bitwise_and(np.right_shift(v, 2), popcount.m2)
        v = np.bitwise_and(v + np.right_shift(v, 4), popcount.m4)
        v = np.right_shift(v*popcount.h01, 56)
        return v.sum()

    ma = 0x01001001001001
    mb = 0x84210842108421
    
    def count1s64_32(d):
        v = np.frombuffer(d, dtype=np.uint32)
        c = (  np.mod(np.bitwise_and(np.bitwise_and(v, 0xfff) * popcount.ma, popcount.mb), 0x1f)
             + np.mod(np.bitwise_and(np.right_shift(np.bitwise_and(v, 0xfff000), 12) * popcount.ma, popcount.mb), 0x1f)
             + np.mod(np.bitwise_and(np.right_shift(v, 24) * popcount.ma, popcount.mb), 0x1f)
             )
        return c.sum()

    def gmpy2count1s(d):
        v = np.frombuffer(d, dtype=np.uint64)*mpz(1)
        return sum(gmpy2.popcount(a) for a in v)

# not meant to be random
random.seed(1)
d = bytearray([random.randint(0,255) for i in range(409600)])

print(popcount.count1s(d))
print(popcount.count1sb(d))
print(popcount.count1s64(d))
print(popcount.count1s64_3(d))
print(popcount.count1s64_32(d))
print(popcount.gmpy2count1s(d))

1638868
1638868
1638868
1638868
1638868
1638868


In [3]:
import timeit
from functools import partial

import random
# not meant to be random
random.seed(1)
d = bytearray([random.randint(0,255) for i in range(409600)])

number = 2000

repeat = 3

In [4]:
print(timeit.repeat(partial(popcount.count1s, d), number=number, repeat=repeat))
print(timeit.repeat(partial(popcount.count1sb, d), number=number, repeat=repeat))
print(timeit.repeat(partial(popcount.count1s64, d), number=number, repeat=repeat))
print(timeit.repeat(partial(popcount.count1s64_3, d), number=number, repeat=repeat))
print(timeit.repeat(partial(popcount.count1s64_32, d), number=number, repeat=repeat))
#print(timeit.repeat("popcount.gmpy2count1s(d)", setup=setup, number=number, repeat=repeat)) # *10 than others


[3.719270120956935, 3.565112615993712, 3.56609258800745]
[2.8327555079595186, 2.7163443389581516, 2.887360363965854]
[2.8160818380420096, 2.780999206006527, 2.815441971004475]
[1.7871111790300347, 1.958980483992491, 1.892992699984461]
[10.705752222042065, 9.834577971021645, 10.33591843402246]


Seems count1s64_3 is winner now.